In [33]:
import logging
import os
import random
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.CepsMy1Experiment import CepsMy1Experiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
from src.experiment.FineTuneExperiment import FineTuneExperiment
pd.options.display.max_columns = 999

In [34]:
import sqlite3
conn = sqlite3.connect("FT_CEPS2.db")

In [35]:
df = pd.read_sql_query("SELECT * FROM results", conn)
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:29:39
1,data/TSP/CEPS_benchmark/grid/04.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:29:39
2,data/TSP/CEPS_benchmark/grid/33.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:29:39
3,data/TSP/CEPS_benchmark/rotation/45.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:29:39
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:29:39
...,...,...,...,...,...,...
34924,data/TSP/CEPS_benchmark/cluster/47.tsp,1988960974175372683,3.43,3.43,finetune1,2024-12-16 13:16:54
34925,data/TSP/CEPS_benchmark/expansion/22.tsp,2152573642869061282,0.43,0.43,finetune1,2024-12-16 13:17:06
34926,data/TSP/CEPS_benchmark/expansion/22.tsp,54945704898153008,8.88,8.88,finetune1,2024-12-16 13:17:14
34927,data/TSP/CEPS_benchmark/expansion/22.tsp,1347094451537985230,100.00,10.00,finetune1,2024-12-16 13:17:27


In [36]:
_ = df.loc[df["comment"].str.startswith("test")].pivot_table(index="instance_id", columns="solver_id", values="cost", aggfunc=lambda x: sorted(x)[1])
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_["min"].describe()

count    470.000000
mean       2.347957
std        9.211847
min        0.070000
25%        0.460000
50%        0.910000
75%        1.980000
max      100.000000
Name: min, dtype: float64

In [37]:
df.loc[df["comment"].str.startswith("finetune")].groupby("instance_id")["cost"].min().describe()

count    470.000000
mean       3.361213
std       13.608225
min        0.060000
25%        0.462500
50%        0.920000
75%        1.867500
max      100.000000
Name: cost, dtype: float64

In [34]:
# [2024-12-16 11:07:40] INFO      Fine-tuning time: count    470.000000
# mean      11.888708
# std        3.821147
# min        8.774319
# 25%       10.217788
# 50%       10.686284
# 75%       11.229128
# max       29.632534
# dtype: float64

In [35]:
# [2024-12-16 10:53:57] INFO      Fine-tuning time: count    470.000000
# mean      11.468917
# std        3.113127
# min        8.793470
# 25%       10.282021
# 50%       10.606351
# 75%       11.195219
# max       24.427696
# dtype: float64

In [36]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [37]:
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance
experiment = FineTuneExperiment(
    db_name="CEPS1.db",
    solver_class=solver_class,
    instance_class=instance_class,
)
best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-16 11:34:17] INFO      [FINE_TUNE] Start!


In [4]:
experiment.fine_tune_all(best_portfolio, train_instances, comment="finetune1")

[2024-12-15 23:51:58] INFO      EPM predicted costs: [2.92828829 5.54125121 3.14796894 3.21887054]
[2024-12-15 23:51:58] INFO      EPM improving solver 1/4
[2024-12-15 23:51:58] DEBUG     solver 2220588739511820280 not cached
[2024-12-15 23:51:58] DEBUG     solver 1888347946418402467 not cached
[2024-12-15 23:51:59] DEBUG     solver 2133623553045416188 not cached
[2024-12-15 23:51:59] DEBUG     solver 1544122698081815385 not cached
[2024-12-15 23:51:59] DEBUG     solver 1507807693254370155 not cached
[2024-12-15 23:51:59] DEBUG     solver 2053552427548661639 not cached
[2024-12-15 23:51:59] DEBUG     solver 1212263318994698480 not cached
[2024-12-15 23:51:59] DEBUG     solver 1153701089608456522 not cached
[2024-12-15 23:51:59] DEBUG     solver 1823403730061373967 not cached
[2024-12-15 23:52:00] DEBUG     solver 618127700742984342 not cached
[2024-12-15 23:52:00] DEBUG     solver 340153343601167199 not cached
[2024-12-15 23:52:00] DEBUG     solver 1854519867917337917 not cached
[2024-

In [4]:
epm = experiment._get_epm()

In [5]:
instance = test_instances[26]
instance.filepath

WindowsPath('C:/Users/zakrz/Documents/DataScience/praca magisterska/raw-algorithm-portfolios/data/TSP/CEPS_benchmark/compression/18.tsp')

In [5]:
instance = [i for i in test_instances if i.__hash__() == "data/TSP/CEPS_benchmark/expansion/02.tsp"][0]

In [6]:
from src.database import db_connect
conn = db_connect()
df_instances = pd.read_sql_query("SELECT * FROM instances WHERE id = 'data/TSP/CEPS_benchmark/expansion/02.tsp' ", conn)
df_instances



,id,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,centroid_centroid_x,centroid_centroid_y,centroid_dist_min,centroid_dist_median,centroid_dist_mean,centroid_dist_max,centroid_dist_sd,centroid_dist_span,centroid_dist_coef_of_var,cluster_01pct_number_of_clusters,cluster_01pct_mean_distance_to_centroid,cluster_05pct_number_of_clusters,cluster_05pct_mean_distance_to_centroid,cluster_10pct_number_of_clusters,cluster_10pct_mean_distance_to_centroid,bounding_box_10_ratio_of_cities_outside_box,bounding_box_20_ratio_of_cities_outside_box,bounding_box_30_ratio_of_cities_outside_box,chull_area,chull_points_on_hull,distance_distances_shorter_mean_distance,distance_distinct_distances,distance_mode_frequency,distance_mode_quantity,distance_mode_mean,distance_mean_tour_length,distance_sum_of_lowest_edge_values,distance_min,distance_median,distance_mean,distance_max,distance_sd,distance_span,distance_coef_of_var,modes_number,mst_depth_min,mst_depth_median,mst_depth_mean,mst_depth_max,mst_depth_sd,mst_depth_span,mst_depth_coef_of_var,mst_dists_min,mst_dists_median,mst_dists_mean,mst_dists_max,mst_dists_sd,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var,num_nodes,cost_matrix_avg,cost_matrix_std,cost_matrix_skew,stdTime,mst_length,mst_length_avg,mst_length_std,mst_length_skew,mst_degree_avg,mst_degree_std,mst_degree_skew,mstTime,cluster_distance_avg,cluster_distance_std,cluster_distance_skew,clusterTime,tour_const_heu_avg,tour_const_std,tour_const_skew,ls_impov_per_step_avg,ls_impov_per_step_std,ls_impov_per_step_skew,ls_steps_2lm_avg,ls_steps_2lm_std,ls_steps_2lm_skew,ls_maxdist_avg,ls_maxdist_std,ls_maxdist_skew,ls_bestsol_avg,ls_bestsol_std,ls_bestsol_skew,ls_backbone_avg,ls_backbone_std,ls_backbone_skew,lpTime,bc_improv_per_cut_avg,bc_improv_per_cut_std,bc_improv_per_cut_skew,bc_upper_lower_ratio,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,data/TSP/CEPS_benchmark/expansion/02.tsp,0.00008,1.538113,1.569852,3.141414,0.898406,3.141334,0.572287,0.615083,0.564333,0.008239,0.260412,0.270169,0.774164,0.13678,0.765925,0.506276,5.0,0.181273,2.0,0.189004,2.0,0.189998,0.0825,0.4825,0.83125,0.67982,0.01625,0.531038,1.0,800.0,0.000002,0.0,596.186285,7.432779,0.0,0.347584,0.372151,1.203049,0.211795,1.203049,0.569111,2.0,1.0,5.0,16.37,90.0,22.111858,89.0,1.350755,0.000469,0.011588,0.013845,0.391967,0.0167,0.391498,1.20625,0.000046,0.000469,0.008535,0.010603,0.106172,0.009176,0.105703,0.865333,800.0,355080.162107,0.56568,3.238406,0.03,0.037166,0.037212,1.206817,0.066178,1.9975,0.339539,6.789576,0.02,0.345205,1.29404,1.061017,1.22,0.05272,1.900000e-08,0.0,0.00371,0.002028,0.733271,0.09875,0.013086,-0.870828,0.1495,0.006782,2.042887,35.096494,0.01427,0.680471,0.810537,0.296091,-0.782259,1.53,0.010825,3.424261,0.711797,0.010944,0.125,0.5,0.5,0.5,0.875,0.9025,0.1975,0.1,417.868816,0.36


In [7]:
experiment.fine_tune(epm, best_portfolio, instance)

[2024-12-15 23:40:58] INFO      EPM predicted costs: [2.99289099 4.97417477 3.21309203 3.30750206]
[2024-12-15 23:40:58] INFO      EPM improving solver 1/4
[2024-12-15 23:40:58] DEBUG     SMAC configuration with surrogate model
[2024-12-15 23:40:58] DEBUG     solver 2220588739511820280 not cached
[2024-12-15 23:40:58] DEBUG     (0) EPM cost: 100.00
[2024-12-15 23:40:58] DEBUG     SMAC iteration 1, cost: 100.00
[2024-12-15 23:40:59] DEBUG     solver 539290601180460939 not cached
[2024-12-15 23:40:59] DEBUG     (0) EPM cost: 100.00
[2024-12-15 23:40:59] DEBUG     SMAC iteration 2, cost: 100.00
[2024-12-15 23:40:59] DEBUG     solver 743791062141100996 not cached
[2024-12-15 23:40:59] DEBUG     (0) EPM cost: 100.00
[2024-12-15 23:40:59] DEBUG     SMAC iteration 3, cost: 100.00
[2024-12-15 23:40:59] DEBUG     solver 974520441262238054 not cached
[2024-12-15 23:40:59] DEBUG     (0) EPM cost: 100.00
[2024-12-15 23:40:59] DEBUG     SMAC iteration 4, cost: 100.00
[2024-12-15 23:40:59] DEBUG    

1.73

In [5]:
from src.database import db_connect
conn = db_connect()

In [6]:
df = pd.read_sql_query("SELECT * FROM results", conn)

In [8]:
df.loc[df["comment"].str.startswith("test1")].sort_values("solver_id")

,instance_id,solver_id,cost,time,comment,created_at
32772,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,1191837438205487429,100.00,10.00,test1,2024-12-15 22:54:50
28340,data/TSP/CEPS_benchmark/grid/30.tsp,1191837438205487429,0.96,0.96,test1,2024-12-13 11:35:39
27556,data/TSP/CEPS_benchmark/expansion/44.tsp,1191837438205487429,7.09,7.09,test1,2024-12-13 11:29:28
28336,data/TSP/CEPS_benchmark/compression/25.tsp,1191837438205487429,0.22,0.22,test1,2024-12-13 11:35:39
28332,data/TSP/CEPS_benchmark/linearprojection/28.tsp,1191837438205487429,0.79,0.79,test1,2024-12-13 11:35:30
...,...,...,...,...,...,...
28414,data/TSP/CEPS_benchmark/uniform_portgen/30.tsp,686356820175302440,4.56,4.56,test1,2024-12-13 11:36:13
27502,data/TSP/CEPS_benchmark/rotation/06.tsp,686356820175302440,3.14,3.14,test1,2024-12-13 11:28:56
28398,data/TSP/CEPS_benchmark/linearprojection/21.tsp,686356820175302440,3.63,3.63,test1,2024-12-13 11:36:08
28062,data/TSP/CEPS_benchmark/explosion/17.tsp,686356820175302440,4.66,4.66,test1,2024-12-13 11:33:25


In [11]:
df.loc[df["comment"].str.startswith("test") & (df["instance_id"] == 'data/TSP/CEPS_benchmark/expansion/02.tsp')].sort_values("solver_id")

,instance_id,solver_id,cost,time,comment,created_at
28248,data/TSP/CEPS_benchmark/expansion/02.tsp,1191837438205487429,100.0,10.0,test1,2024-12-13 11:34:57
30128,data/TSP/CEPS_benchmark/expansion/02.tsp,1191837438205487429,100.0,10.0,test2,2024-12-13 11:49:51
32008,data/TSP/CEPS_benchmark/expansion/02.tsp,1191837438205487429,100.0,10.0,test3,2024-12-13 12:04:43
28245,data/TSP/CEPS_benchmark/expansion/02.tsp,2152573642869061282,100.0,10.0,test1,2024-12-13 11:34:57
30125,data/TSP/CEPS_benchmark/expansion/02.tsp,2152573642869061282,100.0,10.0,test2,2024-12-13 11:49:50
32005,data/TSP/CEPS_benchmark/expansion/02.tsp,2152573642869061282,100.0,10.0,test3,2024-12-13 12:04:43
28247,data/TSP/CEPS_benchmark/expansion/02.tsp,508943627851605191,100.0,10.0,test1,2024-12-13 11:34:57
30127,data/TSP/CEPS_benchmark/expansion/02.tsp,508943627851605191,100.0,10.0,test2,2024-12-13 11:49:51
32007,data/TSP/CEPS_benchmark/expansion/02.tsp,508943627851605191,100.0,10.0,test3,2024-12-13 12:04:43
28246,data/TSP/CEPS_benchmark/expansion/02.tsp,686356820175302440,100.0,10.0,test1,2024-12-13 11:34:57


In [12]:
df.loc[df["instance_id"] == 'data/TSP/CEPS_benchmark/expansion/02.tsp']["comment"]

28245    test1
28246    test1
28247    test1
28248    test1
30125    test2
30126    test2
30127    test2
30128    test2
32005    test3
32006    test3
32007    test3
32008    test3
32753    after
32754    after
32755    after
32756    after
Name: comment, dtype: object

In [ ]:
# t_c = 60
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 2

# experiment = CepsExperiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 1

# experiment = CepsMy1Experiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [22]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [4]:
from src.database import db_connect
conn = db_connect()

In [5]:
import sqlite3
conn = sqlite3.connect("_archive/7/CEPS1.db")
# conn = sqlite3.connect("database/2024_12_15_20_41_04.db")

In [6]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,2220588739511820280,52,1,YES,ALPHA,7,QUADRANT,NO,YES,1,QUICK-BORUVKA,0.582020,3,0,954863081,10,3,3,1,8,YES,YES,6,NO
1,2051815471081033189,45,0,YES,DELAUNAY,10,QUADRANT,YES,YES,3,SIERPINSKI,0.842342,3,5,1010150627,9,6,3,1,26,YES,NO,4,NO
2,574052600841816340,58,1,YES,QUADRANT,6,QUADRANT,NO,YES,5,GREEDY,0.569101,2,5,737143853,6,2,3,4,37,NO,NO,6,YES
3,1333412370196243425,58,1,YES,QUADRANT,6,QUADRANT,YES,YES,1,BORUVKA,0.788739,3,4,1321047180,5,4,4,2,28,YES,NO,0,NO
4,607108495934423959,42,0,YES,DELAUNAY,3,QUADRANT,YES,YES,2,GREEDY,0.972919,4,0,143733986,2,4,1,4,53,NO,YES,5,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,859422855629046700,44,0,YES,QUADRANT,0,QUADRANT,YES,NO,2,NEAREST-NEIGHBOR,0.519511,3,0,397383577,3,5,3,2,80,NO,NO,2,YES
534,927684278101485936,49,1,NO,NEAREST-NEIGHBOR,5,QUADRANT,YES,NO,2,QUICK-BORUVKA,0.260675,1,4,1753781529,1,5,0,3,80,YES,YES,6,NO
535,2044657498304677364,40,0,NO,DELAUNAY,3,QUADRANT,YES,NO,1,QUICK-BORUVKA,0.034273,3,0,853884415,5,3,3,3,32,YES,YES,0,NO
536,66469343800519890,51,1,YES,NEAREST-NEIGHBOR,3,QUADRANT,NO,YES,5,BORUVKA,0.614997,2,0,939551573,7,6,2,1,26,NO,YES,6,YES


In [7]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
...,...
729,data/TSP/CEPS_benchmark/implosion/23.tsp
730,data/TSP/CEPS_benchmark/expansion/42.tsp
731,data/TSP/CEPS_benchmark/compression/17.tsp
732,data/TSP/CEPS_benchmark/cluster/47.tsp


In [8]:
df = pd.read_sql_query("SELECT * FROM results", conn)
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:25:48
1,data/TSP/CEPS_benchmark/grid/04.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:25:48
2,data/TSP/CEPS_benchmark/grid/33.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:25:48
3,data/TSP/CEPS_benchmark/rotation/45.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:25:48
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,2220588739511820280,100.00,10.00,initialization,2024-12-12 21:25:48
...,...,...,...,...,...,...
32748,data/TSP/CEPS_benchmark/cluster/47.tsp,1191837438205487429,3.22,3.22,test3,2024-12-13 12:10:19
32749,data/TSP/CEPS_benchmark/expansion/22.tsp,2152573642869061282,0.71,0.71,test3,2024-12-13 12:10:19
32750,data/TSP/CEPS_benchmark/expansion/22.tsp,686356820175302440,100.00,10.00,test3,2024-12-13 12:10:23
32751,data/TSP/CEPS_benchmark/expansion/22.tsp,508943627851605191,1.01,1.01,test3,2024-12-13 12:10:23


In [9]:
df["comment"].value_counts()

configuration     23284
initialization     2880
test1              1880
test2              1880
test3              1880
mutation            896
validation           53
Name: comment, dtype: int64

In [10]:
_ = df.loc[df["comment"].str.contains("test")].pivot_table(index=["instance_id", "solver_id"], columns="comment", values="time").sort_values(["instance_id", "solver_id"])
_["range"] = _.max(axis=1) - _.min(axis=1)
_.sort_values(by="range")

comment                                                              test1  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/cluster/00.tsp          1191837438205487429  10.00   
data/TSP/CEPS_benchmark/compression/45.tsp      686356820175302440   10.00   
data/TSP/CEPS_benchmark/linearprojection/22.tsp 508943627851605191   10.00   
                                                2152573642869061282  10.00   
data/TSP/CEPS_benchmark/linearprojection/20.tsp 686356820175302440   10.00   
...                                                                    ...   
data/TSP/CEPS_benchmark/uniform_portgen/19.tsp  2152573642869061282   7.55   
data/TSP/CEPS_benchmark/cluster/04.tsp          1191837438205487429   5.28   
data/TSP/CEPS_benchmark/uniform_portgen/37.tsp  508943627851605191    4.16   
data/TSP/CEPS_benchmark/expansion/40.tsp        686356820175302440    7.72   
data/TSP/CEPS_benchmark/linearprojection/38.tsp 686356820175302440    7.11   

comment                                                              test2  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/cluster/00.tsp          1191837438205487429  10.00   
data/TSP/CEPS_benchmark/compression/45.tsp      686356820175302440   10.00   
data/TSP/CEPS_benchmark/linearprojection/22.tsp 508943627851605191   10.00   
                                                2152573642869061282  10.00   
data/TSP/CEPS_benchmark/linearprojection/20.tsp 686356820175302440   10.00   
...                                                                    ...   
data/TSP/CEPS_benchmark/uniform_portgen/19.tsp  2152573642869061282   8.31   
data/TSP/CEPS_benchmark/cluster/04.tsp          1191837438205487429   5.20   
data/TSP/CEPS_benchmark/uniform_portgen/37.tsp  508943627851605191    3.33   
data/TSP/CEPS_benchmark/expansion/40.tsp        686356820175302440    8.59   
data/TSP/CEPS_benchmark/linearprojection/38.tsp 686356820175302440    6.65   

comment                                                              test3  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/cluster/00.tsp          1191837438205487429  10.00   
data/TSP/CEPS_benchmark/compression/45.tsp      686356820175302440   10.00   
data/TSP/CEPS_benchmark/linearprojection/22.tsp 508943627851605191   10.00   
                                                2152573642869061282  10.00   
data/TSP/CEPS_benchmark/linearprojection/20.tsp 686356820175302440   10.00   
...                                                                    ...   
data/TSP/CEPS_benchmark/uniform_portgen/19.tsp  2152573642869061282   7.74   
data/TSP/CEPS_benchmark/cluster/04.tsp          1191837438205487429   5.99   
data/TSP/CEPS_benchmark/uniform_portgen/37.tsp  508943627851605191    3.58   
data/TSP/CEPS_benchmark/expansion/40.tsp        686356820175302440    7.74   
data/TSP/CEPS_benchmark/linearprojection/38.tsp 686356820175302440    6.17   

comment                                                              range  
instance_id                                     solver_id                   
data/TSP/CEPS_benchmark/cluster/00.tsp          1191837438205487429   0.00  
data/TSP/CEPS_benchmark/compression/45.tsp      686356820175302440    0.00  
data/TSP/CEPS_benchmark/linearprojection/22.tsp 508943627851605191    0.00  
                                                2152573642869061282   0.00  
data/TSP/CEPS_benchmark/linearprojection/20.tsp 686356820175302440    0.00  
...                                                                    ...  
data/TSP/CEPS_benchmark/uniform_portgen/19.tsp  2152573642869061282   0.76  
data/TSP/CEPS_benchmark/cluster/04.tsp          1191837438205487429   0.79  
data/TSP/CEPS_benchmark/uniform_portgen/37.tsp  508943627851605191    0.83  
data/TSP/CEPS_benchmark/expansion/40.tsp        686356820175302440    0.87  
data/TSP/CEPS_benchmark/linearp

In [15]:
_ = df.loc[df["comment"].str.startswith("test")].pivot_table(index="instance_id", columns="solver_id", values="cost", aggfunc=lambda x: sorted(x)[1])
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,1191837438205487429,2152573642869061282,508943627851605191,686356820175302440,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,100.00,6.03,3.73,3.50,3,3.50
data/TSP/CEPS_benchmark/cluster/01.tsp,0.77,1.31,2.44,2.79,0,0.77
data/TSP/CEPS_benchmark/cluster/02.tsp,100.00,3.80,3.66,1.89,3,1.89
data/TSP/CEPS_benchmark/cluster/03.tsp,4.19,3.81,3.90,8.13,1,3.81
data/TSP/CEPS_benchmark/cluster/04.tsp,5.28,100.00,100.00,7.07,0,5.28
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,0.34,3.41,0.32,5.44,2,0.32
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,4.24,0.48,2.70,1.02,1,0.48
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,0.20,4.72,0.25,0.46,0,0.20


In [16]:
_["min"].describe()

count    470.000000
mean       1.892681
std        9.202442
min        0.040000
25%        0.320000
50%        0.550000
75%        1.170000
max      100.000000
Name: min, dtype: float64

In [17]:
(_["min"] > 90).value_counts()

False    466
True       4
Name: min, dtype: int64

In [18]:
_.loc[_["min"] > 90]

solver_id,1191837438205487429,2152573642869061282,508943627851605191,686356820175302440,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/09.tsp,100.0,100.0,100.0,100.0,0,100.0
data/TSP/CEPS_benchmark/expansion/02.tsp,100.0,100.0,100.0,100.0,0,100.0
data/TSP/CEPS_benchmark/expansion/06.tsp,100.0,100.0,100.0,100.0,0,100.0
data/TSP/CEPS_benchmark/linearprojection/12.tsp,100.0,100.0,100.0,100.0,0,100.0


In [41]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

3    160
0    129
1    123
2     54
Name: best, dtype: int64

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index